In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

from pathlib import Path
from scipy.interpolate import interp1d

In [2]:
def cargar_senal(path_csv):
    df = pd.read_csv(path_csv, sep=';')
    df.columns = [col.strip() for col in df.columns]
    return df

In [3]:
def upsample_rom(df):
    # Obtener el número total de filas (filas de datos)
    original_len = len(df)
    
    # Contar las celdas no vacías en la columna de ángulos (ROM)
    no_vacias_angulos = df.iloc[:, 1].notna().sum()
    
    # Si hay valores válidos, se puede hacer upsampling
    if no_vacias_angulos == 0:
        raise ValueError("No hay valores válidos en la columna de ángulos.")
    
    # Calcular el factor de upsampling
    factor_upsampling = original_len / no_vacias_angulos

    #ROM Sampling Rate = 20 Hz

    sampling_rate = factor_upsampling * 20
    
    # Crear el espacio de interpolación para ROM
    x_original = np.linspace(0, 1, no_vacias_angulos)
    x_new = np.linspace(0, 1, original_len)
    
    # Crear DataFrame vacío para los datos upsampled
    upsampled_df = df.copy()  # Copiar el DataFrame original para no modificar las otras columnas
    
    # Interpolación solo de la columna de ROM
    #rom_col = 'angles_biceps_brachii_(right)_triceps_brachii_long_(right)'
    interpolator = interp1d(x_original, df.iloc[:, 1].dropna(), kind='cubic', fill_value='extrapolate')
    upsampled_df.iloc[:, 1] = interpolator(x_new)
    
    return upsampled_df, sampling_rate

In [4]:
def calcular_aceleracion(df, sampling_rate):
    #rom_col = 'angles_biceps_brachii_(right)_triceps_brachii_long_(right)'
    tiempo = np.arange(len(df)) / sampling_rate
    
    d_speed = np.diff(df.iloc[:, 1]) / np.diff(tiempo)
    d_speed = np.insert(d_speed, 0, d_speed[0])
    
    d_acceleration = np.diff(d_speed) / np.diff(tiempo)
    d_acceleration = np.insert(d_acceleration, 0, d_acceleration[0])
    
    d_jerk = np.diff(d_acceleration) / np.diff(tiempo)
    d_jerk = np.insert(d_jerk, 0, d_jerk[0])

    #Save in the dataframe only coherent values (fix first values of acceleration and jerk)
    
    d_acceleration[0] = d_acceleration[2]
    d_acceleration[1] = d_acceleration[2]

    d_jerk[0] = d_jerk[3]
    d_jerk[1] = d_jerk[3]
    d_jerk[2] = d_jerk[3]
    
    df['speed'] = d_speed
    df['acceleration'] = d_acceleration
    df['jerk'] = d_jerk
    
    return df

In [5]:
def resolver_offsets_emg(df):
    for col in df.columns:
        if 'emg_muscle' in col:
            offset = df[col].mean()
            df[col] = df[col] - offset
    return df

In [6]:
def normalizar_tiempo(df):
    n = len(df)
    df['tiempo_normalizado'] = np.linspace(0, 1, n)
    return df

In [7]:
def columnas_identificadoras(df, path):
    carpetas = os.path.dirname(path).split('/')
    #print(carpetas)
    del carpetas[0]
    #print(carpetas)
    df['class'] = [carpetas[0]] * len(df)
    bpm = re.findall(r'\d+', carpetas[1])
    df['BPM'] = [int(bpm[0])] * len(df)
    df['kid number'] = [carpetas[2]] * len(df)
    return df

In [8]:
def borrar_y_renombrar(df):
    df = df.drop(['angles_window', 'tiempo_normalizado'], axis = 1)
    df.columns = ['ROM', 'Biceps EMG', 'Triceps EMG', 'Speed', 'Acceleration', 'Jerk', 'Class', 'BPM', 'Kid number']
    return df

In [9]:
def añadir_a_tabla_global(global_df, df):
    global_df.loc[len(global_df)] = [
        df['ROM'].tolist(),
        df['Biceps EMG'].tolist(),
        df['Triceps EMG'].tolist(),
        df['Speed'].tolist(),
        df['Acceleration'].tolist(),
        df['Jerk'].tolist(),
        df['Class'].iloc[0],
        df['BPM'].iloc[0],
        df['Kid number'].iloc[0]
    ]
    return global_df

In [10]:
columnas = ['ROM', 'Biceps EMG', 'Triceps EMG', 'Speed', 'Acceleration', 'Jerk', 'Class', 'BPM', 'Kid number']
global_df = pd.DataFrame(columns=columnas)

In [11]:
#path = 'MEDICIONES_COLEGIO/2ESO/42BPM/1/advanced_mdurance-test-240835 (5).csv'
root = Path('MEDICIONES_COLEGIO')
for csv_file in root.rglob('advanced_*.csv'):
    if '.ipynb_checkpoints' in csv_file.parts:
        continue
    path = csv_file.as_posix()
    #print(path)
    df = cargar_senal(path)
    df_copy = df.copy()
    df, sampling_rate = upsample_rom(df)
    df = calcular_aceleracion(df, sampling_rate)
    df = resolver_offsets_emg(df)
    df = normalizar_tiempo(df)
    df = columnas_identificadoras(df, path)
    df = borrar_y_renombrar(df)
    global_df = añadir_a_tabla_global(global_df, df)
    df = None

In [12]:
global_df

,ROM,Biceps EMG,Triceps EMG,Speed,Acceleration,Jerk,Class,BPM,Kid number
0,"[69.89, 69.88576561129825, 69.8814602272893, 6...","[-464.7238445339335, -437.02884453393335, -403...","[-18.156588653321705, -18.156588653321705, -15...","[-4.230880924123889, -4.230880924123889, -4.30...","[-70.87773057127735, -70.87773057127735, -70.8...","[956.7065392567317, 956.7065392567317, 956.706...",2ESO,42,1
1,"[72.71, 72.70922850195431, 72.70843108503699, ...","[-1849.6745578969837, -1861.9835578969837, -18...","[-741.7798868591992, -717.1618868591993, -707....","[-0.7248765735209701, -0.7248765735209701, -0....","[-22.880976729017448, -22.880976729017448, -22...","[222.72032670630173, 222.72032670630173, 222.7...",2ESO,42,10
2,"[69.45, 69.57506082770446, 69.69964192977204, ...","[1452.580722296214, 1449.5037222962137, 1483.3...","[1482.2336971270147, 1463.7696971270148, 1509....","[125.03024217594448, 125.03024217594448, 124.5...","[-479.4910171630069, -479.4910171630069, -479....","[1479.1549884470062, 1479.1549884470062, 1479....",2ESO,42,11
3,"[54.36, 54.36553355428835, 54.37088841485495, ...","[-1064.4384637935202, -1073.6704637935202, -10...","[-409.6858790498072, -415.840879049807, -415.8...","[5.201541031047441, 5.201541031047441, 5.03356...","[-157.89377253226746, -157.89377253226746, -15...","[1187.0361555220463, 1187.0361555220463, 1187....",2ESO,42,12
4,"[68.52, 68.51296899626853, 68.50598273848837, ...","[1711.3306176096203, 1852.8856176096203, 1945....","[-1639.0274882593512, -1666.722488259351, -165...","[-7.025866056212713, -7.025866056212713, -6.98...","[44.680581934775034, 44.680581934775034, 44.68...","[-666.7064848088461, -666.7064848088461, -666....",2ESO,42,13
...,...,...,...,...,...,...,...,...,...
352,"[76.84, 76.83638809250162, 76.83288963006542, ...","[-709.0967537104198, -706.0197537104195, -699....","[145.00659841489414, 145.00659841489414, 151.1...","[-3.4629001316920784, -3.4629001316920784, -3....","[104.27790381829415, 104.27790381829415, 104.2...","[-744.5497644366299, -744.5497644366299, -744....",5PRIMARIA,78,5
353,"[67.09, 67.0908884528519, 67.09176512077997, 6...","[210.86569965815625, 189.32469965815653, 186.2...","[48.34200170921895, 42.18800170921895, 51.4190...","[0.8337651274787337, 0.8337651274787337, 0.822...","[-10.37875981335117, -10.37875981335117, -10.3...","[49.28840490441344, 49.28840490441344, 49.2884...",5PRIMARIA,78,6
354,"[52.03, 52.03021718290898, 52.030418182060984,...","[-55.090403494130214, -144.33240349413018, -11...","[-98.68055216157586, -70.9855521615757, -89.44...","[0.20383438240352109, 0.20383438240352109, 0.1...","[-14.255515939735364, -14.255515939735364, -14...","[371.66992084380934, 371.66992084380934, 371.6...",5PRIMARIA,78,7
355,"[49.83, 49.83125563872093, 49.832583830490115,...","[-1169.9726537759525, -1157.6636537759532, -11...","[-17.185382272416973, -26.417382272416944, -26...","[1.179313824397575, 1.179313824397575, 1.24745...","[64.00074700568724, 64.00074700568724, 64.0007...","[-898.9812588715627, -898.9812588715627, -898....",5PRIMARIA,78,8


In [13]:
global_df.to_excel('global_table.xlsx', index=False)